In [52]:
import pandas as pd
import numpy as np
import glob
import sys
import pywt


# Fins all cnv files based on entity name
GLOB_PATH = "/home/alpha/programs/python_files/datasets/cnv_and_mut/"
ENTITY_NAME = ["gbm","k27","mng","oligo","pxa","astroLow","astroHigh","pa"]
FILE_TYPE = input("which file type to use? wes/igv \n")

if FILE_TYPE == "wes":
    POSTFIX = "_cnv/*.MAXDENS.segments.seg"
elif FILE_TYPE == "igv":
    POSTFIX = "_cnv/*.bins.igv"

# add all paths to a list
ENTITY_NAME_PATH = []
for entity in ENTITY_NAME:
    entityPath = glob.glob(GLOB_PATH + entity + POSTFIX)
    ENTITY_NAME_PATH.extend(entityPath)

# read the files into a dataframe
df_cnv = pd.DataFrame()
for entity in ENTITY_NAME_PATH:
    if FILE_TYPE == "seg":
        file = pd.read_csv(entity,sep="\t",header=0,names=["ID","chr","start","end","mark","bstat","pval","seg_mean","seg_median","alteration"])
        file["entity"] = entity.split("/")[-2]
        file["chr"] = file["chr"].replace(regex=r'[a-z]',value = '')
        file["chr"] = file["chr"].replace(regex=[r"X","Y"],value=[23,24])
        file["chr"] = file["chr"].astype("int")
    elif FILE_TYPE == "igv":
        file = pd.read_csv(entity,sep="\t",header=0,names=["chr","start","end","feature","value"])
        file = file[["feature",file.columns.tolist()[-1]]]
        file = file.set_index("feature")
        # file["value"] = 2*((file["value"]-file["value"].min())/(file["value"].max()-file["value"].min()))-1
        # file["value"] = (file["value"]-file["value"].mean())/file["value"].std()
        decomp_function = pd.Series(pywt.wavedec(file["value"], 'db1', level=2)[0])
        decomp_function = decomp_function.to_frame()
        decomp_function = decomp_function.T
        # extract the entity name from the path
        decomp_function["entity"] = entity.split("/")[-2] 
        # extract the ID from the path
        decomp_function["ID"] = entity.split("/")[-1].split(".")[0]
        # print(decomp_function.shape,decomp_function)
        # print(len(decomp_function))
        if decomp_function.shape == (1,3883):
            df_cnv = pd.concat([df_cnv,decomp_function])

df_cnv = df_cnv.set_index("ID")   
print(df_cnv.tail(10))

answer = input("do you what to save .cvs file? y/n \n")
if answer == "y":
    df_cnv.to_csv(GLOB_PATH + "entities_transformed.csv")
else:
    print("exiting program")

which file type to use? wes/igv 
 igv


                          0       1       2       3       4       5       6  \
ID                                                                            
205059710078_R06C01  0.0595 -0.1125 -0.1755 -0.1025 -0.0905 -0.0405 -0.2590   
204332530056_R03C01  0.0700 -0.0120 -0.0090 -0.0280  0.0115  0.0150 -0.1140   
205857060122_R03C01  0.0545 -0.0560 -0.0250 -0.0180  0.0165 -0.0080 -0.0500   
206065910023_R01C01  0.1110 -0.0190 -0.0780 -0.0510 -0.0085 -0.0200 -0.0445   
206466480047_R08C01  0.1020 -0.0655 -0.1805 -0.1100 -0.0730  0.0030 -0.0845   
205857060122_R04C01  0.0285 -0.1215 -0.1395 -0.1015  0.0190 -0.0805 -0.2280   
205292900047_R04C01  0.0530 -0.0830 -0.1460 -0.1260 -0.0455 -0.0760 -0.1600   
205061430120_R05C01  0.0300  0.0535 -0.0280 -0.0075 -0.1170 -0.0330 -0.1165   
205828600042_R08C01  0.0630 -0.1510 -0.1780 -0.0760 -0.0280 -0.0750 -0.1920   
206600570023_R02C01  0.0410 -0.0635 -0.1290 -0.0420 -0.0530 -0.0190 -0.1745   

                          7       8       9  ...   

do you what to save .cvs file? y/n 
 y


In [71]:
# plot cnv
fig = px.scatter(
    df2, x=x2, y=y2, color="chrom", color_continuous_scale="rdylgn"
)

fig.update_traces(marker_size=3)
fig.update_layout(width=1500, height=800)

fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['ID', 'chrom', 'loc.start', 'loc.end', 'num.mark', 'bstat', 'pval', 'seg.mean', 'seg.median', 'alteration'] but received: 84